# Importing the dataset

In [1]:
import pandas as pd 
import numpy as np
import pandasql as ps
import os
import re
import random 
import numpy as np
import warnings
import tensorflow as tf 
from imblearn.over_sampling import RandomOverSampler
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from preprocess_days_stats import preprocess_match_days
from preprocess_time_serie import preprocess_teams, create_time_series_features
from preprocess_time_series_features import preprocess_features_time_series
from tensorflow.keras import layers
from helper_functions_tensorflow import CSVLoggerCallback, CSVLoggerCallbackParams, plot_loss_curve

# Ignora tutti i warning temporaneamente
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

df_giornate = preprocess_match_days(r"c:\Users\Hp\Documents\Serie_A_dump\csv_serie_a")

Reading file: I1 (0).csv
Reading file: I1 (1).csv
Reading file: I1 (10).csv
Reading file: I1 (11).csv
Reading file: I1 (12).csv
Reading file: I1 (13).csv
Reading file: I1 (14).csv
Reading file: I1 (15).csv
Reading file: I1 (16).csv
Reading file: I1 (17).csv
Reading file: I1 (18).csv
Reading file: I1 (2).csv
Reading file: I1 (3).csv
Reading file: I1 (4).csv
Reading file: I1 (5).csv
Reading file: I1 (6).csv
Reading file: I1 (7).csv
Reading file: I1 (8).csv
Reading file: I1 (9).csv
preprocessing finished!


In [2]:
df_giornate.head()

,div,giornata,stagione,date,hometeam,awayteam,ft_home_goals,ft_away_goals,ft_result,ht_home_goals,ht_away_goals,ht_results,home_shots,away_shots,home_shots_targ,away_shots_targ,home_fouls,away_fouls,home_corners,away_corners,home_yellow,away_yellow,home_red,away_red,home_win_odds,draw_odds,away_win_odds
0,I1,1,2005/2006,2005-08-27,Fiorentina,Sampdoria,2.0,1.0,H,2.0,0.0,H,15.0,9.0,9.0,4.0,18.0,23.0,9.0,3.0,1.0,4.0,0.0,0.0,2.20,2.87,3.25
1,I1,1,2005/2006,2005-08-27,Livorno,Lecce,2.0,1.0,H,1.0,1.0,D,17.0,6.0,7.0,5.0,27.0,21.0,9.0,2.0,2.0,3.0,0.0,0.0,1.90,2.90,4.00
2,I1,1,2005/2006,2005-08-28,Ascoli,Milan,1.0,1.0,D,0.0,0.0,D,8.0,16.0,3.0,9.0,22.0,16.0,0.0,5.0,2.0,1.0,0.0,0.0,7.50,3.75,1.40
3,I1,1,2005/2006,2005-08-28,Inter,Treviso,3.0,0.0,H,1.0,0.0,H,16.0,7.0,9.0,3.0,13.0,20.0,6.0,1.0,1.0,3.0,0.0,0.0,1.16,5.50,13.00
4,I1,1,2005/2006,2005-08-28,Juventus,Chievo,1.0,0.0,H,1.0,0.0,H,16.0,2.0,7.0,0.0,16.0,12.0,6.0,2.0,1.0,2.0,0.0,0.0,1.25,5.00,13.00


# Data preprocessing

In [3]:
all_features = ['ft_goals','ft_goals_conceded','shots','shots_target', 'fouls_done','corners_obtained', 'yellows', 'reds']
less_features = ['ft_goals','ft_goals_conceded','shots', 'fouls_done','corners_obtained', 'reds']
few_features = ['ft_goals','ft_goals_conceded','shots', 'reds']

In [34]:
num_features = 'all'
Statistiche_squadre_dict = preprocess_teams(dataframe = df_giornate)
df_Serie_A = create_time_series_features(num_features, Statistiche_squadre_dict, df_giornate, 4).dropna()
df_Serie_A[(df_Serie_A['hometeam']=='Lecce') | (df_Serie_A['awayteam']=='Lecce')].head(6)

utilizzando tutte le features
preprocess finished


,div,giornata,stagione,date,hometeam,awayteam,home_ft_goals_1,home_ft_goals_2,home_ft_goals_3,home_ft_goals_4,home_ft_goals_conceded_1,home_ft_goals_conceded_2,home_ft_goals_conceded_3,home_ft_goals_conceded_4,home_shots_1,home_shots_2,home_shots_3,home_shots_4,home_shots_target_1,home_shots_target_2,home_shots_target_3,home_shots_target_4,home_fouls_done_1,home_fouls_done_2,home_fouls_done_3,home_fouls_done_4,home_corners_obtained_1,home_corners_obtained_2,home_corners_obtained_3,home_corners_obtained_4,home_yellows_1,home_yellows_2,home_yellows_3,home_yellows_4,home_reds_1,home_reds_2,home_reds_3,home_reds_4,away_ft_goals_1,away_ft_goals_2,away_ft_goals_3,away_ft_goals_4,away_ft_goals_conceded_1,away_ft_goals_conceded_2,away_ft_goals_conceded_3,away_ft_goals_conceded_4,away_shots_1,away_shots_2,away_shots_3,away_shots_4,away_shots_target_1,away_shots_target_2,away_shots_target_3,away_shots_target_4,away_fouls_done_1,away_fouls_done_2,away_fouls_done_3,away_fouls_done_4,away_corners_obtained_1,away_corners_obtained_2,away_corners_obtained_3,away_corners_obtained_4,away_yellows_1,away_yellows_2,away_yellows_3,away_yellows_4,away_reds_1,away_reds_2,away_reds_3,away_reds_4,ft_result,home_win_odds,draw_odds,away_win_odds
43,I1,5,2005/2006,2005-09-25,Empoli,Lecce,3.0,0.0,0.0,0.0,1.0,1.0,4.0,1.0,8.0,16.0,9.0,8.0,4.0,3.0,1.0,4.0,22.0,19.0,16.0,12.0,8.0,7.0,3.0,5.0,3.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,3.0,0.0,2.0,10.0,7.0,9.0,6.0,4.0,5.0,6.0,5.0,18.0,19.0,18.0,21.0,2.0,3.0,9.0,2.0,3.0,1.0,2.0,3.0,1.0,0.0,0.0,0.0,H,2.00,3.20,3.75
55,I1,6,2005/2006,2005-10-02,Lecce,Cagliari,0.0,1.0,0.0,0.0,1.0,3.0,3.0,0.0,12.0,10.0,7.0,9.0,2.0,4.0,5.0,6.0,24.0,18.0,19.0,18.0,5.0,2.0,3.0,9.0,4.0,3.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,3.0,1.0,1.0,8.0,9.0,16.0,9.0,3.0,5.0,8.0,3.0,12.0,16.0,20.0,12.0,2.0,5.0,8.0,6.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,1.0,H,2.10,3.00,3.75
69,I1,7,2005/2006,2005-10-16,Reggina,Lecce,1.0,2.0,0.0,1.0,2.0,0.0,1.0,3.0,8.0,6.0,8.0,16.0,1.0,4.0,3.0,8.0,10.0,20.0,19.0,11.0,1.0,4.0,5.0,5.0,0.0,4.0,4.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,1.0,3.0,3.0,11.0,12.0,10.0,7.0,8.0,2.0,4.0,5.0,24.0,24.0,18.0,19.0,4.0,5.0,2.0,3.0,0.0,4.0,3.0,1.0,0.0,0.0,1.0,0.0,H,2.10,2.80,3.50
73,I1,8,2005/2006,2005-10-23,Lecce,Juventus,0.0,3.0,0.0,1.0,2.0,0.0,1.0,3.0,9.0,11.0,12.0,10.0,4.0,8.0,2.0,4.0,25.0,24.0,24.0,18.0,3.0,4.0,5.0,2.0,4.0,0.0,4.0,3.0,1.0,0.0,0.0,1.0,1.0,2.0,2.0,1.0,0.0,0.0,1.0,0.0,14.0,8.0,8.0,7.0,10.0,5.0,6.0,3.0,24.0,29.0,27.0,18.0,6.0,1.0,5.0,1.0,2.0,2.0,3.0,1.0,0.0,0.0,0.0,0.0,A,7.00,3.75,1.50
86,I1,9,2005/2006,2005-10-26,Palermo,Lecce,1.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,8.0,9.0,21.0,10.0,4.0,6.0,10.0,4.0,18.0,26.0,13.0,21.0,6.0,4.0,11.0,7.0,3.0,3.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,2.0,0.0,1.0,5.0,9.0,11.0,12.0,2.0,4.0,8.0,2.0,30.0,25.0,24.0,24.0,9.0,3.0,4.0,5.0,1.0,4.0,0.0,4.0,0.0,1.0,0.0,0.0,H,1.57,3.50,6.50
94,I1,10,2005/2006,2005-10-30,Lecce,Messina,0.0,0.0,0.0,3.0,3.0,3.0,2.0,0.0,12.0,5.0,9.0,11.0,5.0,2.0,4.0,8.0,16.0,30.0,25.0,24.0,7.0,9.0,3.0,4.0,3.0,1.0,4.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,4.0,14.0,10.0,8.0,18.0,8.0,3.0,4.0,10.0,22.0,16.0,26.0,15.0,3.0,3.0,1.0,5.0,4.0,3.0,4.0,4.0,0.0,1.0,0.0,0.0,A,2.00,3.00,3.50


In [35]:
(Train_teams_encoded, Valid_teams_encoded, Test_teams_encoded, Train_labels_encoded, Valid_labels_encoded, Test_labels_encoded, 
    Train_dict_features_norm, Valid_dict_features_norm, Test_dict_features_norm, 
    Train_teams, Valid_teams, Test_teams, Train_labels, Valid_labels, Test_labels, Train_dict_features, Valid_dict_features, Test_dict_features, 
    Train_odds, Valid_odds, Test_odds) = preprocess_features_time_series(df_Serie_A, num_features)

utilizzando tutte le features
utilizzando tutte le features
utilizzando tutte le features


## Controlliamo che le numeriche tornino tutte

### Caso all features

In [20]:
numero_features = len(all_features)*5*2
numero_features, len(Train_labels), len(Valid_labels), len(Test_labels)

(80, 5104, 720, 301)

In [19]:
for i in (Train_teams_encoded, Valid_teams_encoded, Test_teams_encoded, Train_labels_encoded, Valid_labels_encoded, Test_labels_encoded, 
    ):
    print(f' shape: {i.shape}')

for i in list(Train_dict_features_norm.keys()):
    print(Train_dict_features_norm[i].shape)

 shape: (5104, 78)
 shape: (720, 78)
 shape: (301, 78)
 shape: (5104, 3)
 shape: (720, 3)
 shape: (301, 3)
(720, 10)
(720, 10)
(720, 10)
(720, 10)
(720, 10)
(720, 10)
(720, 10)
(720, 10)


## Creting fast preprocessing datasets

((720, 10), (720, 10))

In [29]:
Array_train_feature = Train_dict_features_norm[list(Train_dict_features_norm.keys())[0]]
for feature in list(Train_dict_features_norm.keys())[1:]:
    Array_train_feature = np.concatenate((Array_train_feature, Train_dict_features_norm[feature]), axis=1)

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [ ]:
# fast preprocessing 
#creo un array con le features concatenate
Array_train_feature = Train_dict_features_norm[list(Train_dict_features_norm.keys())[0]]
for feature in list(Train_dict_features_norm.keys())[1:]:
    Array_train_feature = np.concatenate((Array_train_feature, Train_dict_features_norm[feature]), axis=1)
#creo i fast preprocessing datasets
Dataset_train_norm = tf.data.Dataset.from_tensor_slices((Train_teams_encoded, Array_train_feature))
Train_labels_encoded = tf.data.Dataset.from_tensor_slices(Train_labels_encoded) # make labels
Dataset_train_norm = tf.data.Dataset.zip((Dataset_train_norm, Train_labels_encoded))

#creo un array con le features concatenate
Array_valid_feature = Valid_dict_features_norm[list(Valid_dict_features_norm.keys())[0]]
for feature in list(Valid_dict_features_norm.keys())[1:]:
    Array_valid_feature = np.concatenate((Array_valid_feature, Valid_dict_features_norm[feature]), axis=1)
#creo i fast preprocessing datasets
Dataset_valid_norm = tf.data.Dataset.from_tensor_slices((Valid_teams_encoded, Array_valid_feature))
Valid_labels_encoded = tf.data.Dataset.from_tensor_slices(Valid_labels_encoded) # make labels
Dataset_valid_norm = tf.data.Dataset.zip((Dataset_valid_norm, Valid_labels_encoded))

#creo un array con le features concatenate
Array_test_feature = Test_dict_features_norm[list(Test_dict_features_norm.keys())[0]]
for feature in list(Test_dict_features_norm.keys())[1:]:
    Array_test_feature = np.concatenate((Array_test_feature, Test_dict_features_norm[feature]), axis=1)
#creo i fast preprocessing datasets
Dataset_Test_norm = tf.data.Dataset.from_tensor_slices((Test_teams_encoded, Array_test_feature))
Test_labels_encoded = tf.data.Dataset.from_tensor_slices(Test_labels_encoded) # make labels
Dataset_Test_norm = tf.data.Dataset.zip((Dataset_Test_norm, Test_labels_encoded))

Dataset_train_norm = Dataset_train_norm.batch(32).prefetch(tf.data.AUTOTUNE) #Autotune è per dirgli di prefetchare tanti dati quanti può
Dataset_valid_norm = Dataset_valid_norm.batch(32).prefetch(tf.data.AUTOTUNE)
Dataset_Test_norm = Dataset_Test_norm.batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
Dataset_train_norm

# Model_1: Dense model